### Set up

Library

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from sentence_transformers import SentenceTransformer
from langchain_google_genai import ChatGoogleGenerativeAI
import re
import json
import subprocess
import time
import threading
import psycopg2
import pandas as pd
import numpy as np
import getpass
import os
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler

e:\NhiAri\Code\GroupProject2024\groupproject2024\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


#### postgresSQL

In [ ]:
postgres_url = getpass.getpass("Enter your postgresql url: ")

In [ ]:
model = SentenceTransformer('bkai-foundation-models/vietnamese-bi-encoder')

#### Gemini

In [ ]:
os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

In [ ]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

### Query

#### Get features

##### Hotels

In [ ]:
# amenities
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

cur.execute("SET search_path TO travel_database, public;")

cur.execute("""
    SELECT DISTINCT unnest(amenities) AS unique_amenities
    FROM hotel;
""")

rows = cur.fetchall()

cur.close()
conn.close()

amenities_list = [row[0] for row in rows]
amenities_list_str = "\n    ".join(f'"{amenities_type}"' for amenities_type in amenities_list)
print(amenities_list_str)

"WiFi tại khu vực chung"
    "Money changer"
    "Bathtub"
    "Tiện nghi cho trẻ"
    "Quầy bar bên hồ bơi"
    "Heater"
    "Dù (ô) che nắng"
    "Đưa đón đến khu trượt tuyết (thu phí)"
    "Tiện nghi hội họp"
    "Sân quần vợt ngoài trời"
    "Trung tâm chăm sóc trẻ em"
    "Roll-in shower"
    "Vegetarian meal"
    "Hồ bơi"
    "Tủ lạnh (dùng chung)"
    "Máy photocopy"
    "Giữ trẻ"
    "Porter"
    "Bicycle storage"
    "Dịch vụ cho thuê xe đạp"
    "Conference room"
    "Dịch vụ trông trẻ có người giám hộ"
    "Grocery"
    "Giặt ủi"
    "Dịch vụ phòng 24 giờ"
    "Lò vi sóng"
    "Express check-out"
    "Đưa đón sân bay"
    "AC"
    "Dịch vụ phòng (có giới hạn thời gian)"
    "Bóng quần"
    "TV lounge"
    "A la carte lunch"
    "Bồn tắm nước nóng"
    "Gói cầu hôn lãng mạn"
    "Safety deposit box"
    "Vườn thú bán hoang dã"
    "Hồ bơi trẻ em"
    "Ghế dài tắm nắng"
    "Đưa đón đến trạm xe buýt (thu phí)"
    "Wifi (miễn phí)"
    "Thẩm mỹ viện"
    "Bữa trưa món tự chọn"

In [ ]:
# style
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

cur.execute("SET search_path TO travel_database, public;")

cur.execute("""
    SELECT DISTINCT style
    FROM hotel
    WHERE style IS NOT NULL;
""")

rows = cur.fetchall()

cur.close()
conn.close()
style_list = [row[0] for row in rows]
style_list_str = "\n    ".join(f'"{style}"' for style in style_list)
print(style_list_str)

"Business
"
    "Adventure 
"
    "Romantic 
"
    "Eco-friendly 
"
    "Business 
"
    "Wellness 
"
    "Family-friendly 
"
    "Cultural
"
    "Romantic
"
    "Beachfront 
"
    "Luxury
"
    "Boutique
"
    "Eco-friendly
"
    "Adventure
"
    "Family-friendly
"
    "Boutique 
"
    "Cultural 
"
    "Wellness
"
    "Luxury 
"


##### Attractions

In [ ]:
# types
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

cur.execute("SET search_path TO travel_database, public;")

cur.execute("""
    SELECT DISTINCT unnest(attraction_type) AS unique_attraction_type
    FROM touristattraction;
""")

rows = cur.fetchall()

cur.close()
conn.close()

att_type_list = [row[0] for row in rows]
att_type_list_str = "\n    ".join(f'"{att_type}"' for att_type in att_type_list)
print(att_type_list_str)

"Nhà hát và biểu diễn"
    "Viện bảo tàng lịch sử"
    "Thủy cung"
    "Khu vực đi dạo tham quan di tích lịch sử"
    "Trường đại học và trường học"
    "Quán bar và câu lạc bộ"
    "Khu vực đi dạo ngắm cảnh"
    "Viện bảo tàng nghệ thuật"
    "Vườn"
    "Di tích cổ"
    "Nhà thờ và nhà thờ lớn"
    "ATV và xe địa hình"
    "Đài kỷ niệm và tượng"
    "Cầu"
    "Chuyến tham quan văn hóa"
    "Xưởng vẽ và làm đồ gốm"
    "Núi"
    "Địa điểm giáo dục"
    "Khu liên hợp thể thao"
    "Buổi học và hội thảo"
    "Cửa hàng đồ cổ"
    "Sân gôn"
    "Triển lãm"
    "Đấu trường và sân vận động"
    "Phòng trưng bày nghệ thuật"
    "Điểm thu hút khách tham quan và thắng cảnh"
    "Địa điểm tâm linh"
    "Chợ hoa"
    "Cửa hàng của nhà máy"
    "Trung tâm nghệ thuật"
    "Quán bar rượu vang"
    "Căn cứ và doanh trại quân đội"
    "Địa điểm lịch sử"
    "Công viên nước"
    "Chuyến tham quan cà phê và trà"
    "Nhà hát"
    "Trung tâm trò chơi và giải trí"
    "Bảo t

##### Restaurant

In [ ]:
# Type
# Establish the connection
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

# Set the search path to use the correct schema
cur.execute("SET search_path TO travel_database, public;")

# Query to extract distinct districts from the address composite type
cur.execute("""
    SELECT DISTINCT unnest(restaurant_type) AS unique_res_type
    FROM restaurant;
""")

# Fetch all rows
rows = cur.fetchall()

# Close the cursor and connection
cur.close()
conn.close()

# Convert the rows into a list and format the output
res_type_list = [row[0] for row in rows]
res_type_list_str = "\n    ".join(f'"{res_type}"' for res_type in res_type_list)
print(res_type_list_str)

"Karaoke"
    "Café/Dessert"
    "Buffet"
    "Ăn vặt/vỉa hè"
    "Tiệc cưới/Hội nghị"
    "Quán ăn"
    "Tiệm bánh"
    "Ăn chay"
    "Nhà hàng"


In [ ]:
# Suitable for
# Establish the connection
conn = psycopg2.connect(postgres_url)
cur = conn.cursor()

# Set the search path to use the correct schema
cur.execute("SET search_path TO travel_database, public;")

# Query to extract distinct districts from the address composite type
cur.execute("""
    SELECT DISTINCT unnest(suitable_for) AS unique_res_suit
    FROM restaurant;
""")

# Fetch all rows
rows = cur.fetchall()

# Close the cursor and connection
cur.close()
conn.close()

# Convert the rows into a list and format the output
res_suit_list = [row[0] for row in rows]
res_suit_list_str = "\n    ".join(f'"{res_suit}"' for res_suit in res_suit_list)
print(res_suit_list_str)

"Uống bia - Nhậu"
    "Ăn gia đình"
    "Ăn chay"
    "Ăn Fastfood"
    "Đãi tiệc"
    "Tiếp khách"
    "Takeaway - Mang về"
    "Họp nhóm"
    "Ăn vặt"
    "Nghe nhạc"
    "Du lịch"
    "Ngắm cảnh"
    "Chụp hình - Quay phim"
    "BBQ - Món Nướng"
    "Tiệc ngoài trời"
    "Thư giãn"
    "Hẹn hò"
    "Buffet"


#### Prompt

thêm district

Dựa vào câu prompt này để lấy ra các thông tin yêu cầu về hotel, restaurant, TouristAttraction

##### Extract first JSON

In [ ]:
travel_type_list = ["Nghỉ dưỡng", "Khám phá"]
companion_list = ["friends", "family", "colleagues"]
transport_list = ["self-drive car", "motorbike", "bicycle", "public transport"]
city_list = ["Hà Nội"]
district_list = ["Ba Đình", "Hoàn Kiếm", "Tây Hồ", "Long Biên", "Cầu Giấy", "Đống Đa", "Hai Bà Trưng", "Hoàng Mai", "Thanh Xuân", "Nam Từ Liêm", "Bắc Từ Liêm", "Hà Đông", "Sơn Tây"]

In [ ]:
# template = """
# You are an AI travel suggestion chatbot. Analyze the following travel request:

# Request: "{travel_request}"

# Extract general and specific requirements for Hotels, Restaurants, and Tourist Attractions, even if some are not explicitly mentioned. For each type, provide the following information:

# **General Requirements:**
# - Type: Only assign "Nghỉ dưỡng" or "Khám phá" when the request EXPLICITLY mentions the purpose:
#   * "Nghỉ dưỡng" - ONLY if request contains clear relaxation keywords:
#     Examples that should return "Nghỉ dưỡng":
#     - "Muốn đi nghỉ dưỡng ở resort"
#     - "Tìm chỗ thư giãn cuối tuần"
#     - "Cần resort để nghỉ ngơi"
    
#   * "Khám phá" - ONLY if request contains clear exploration keywords:
#     Examples that should return "Khám phá":
#     - "Muốn đi khám phá văn hóa địa phương"
#     - "Tìm địa điểm để tham quan và trải nghiệm"
#     - "Lên lịch đi phượt và khám phá"
    
#   * Return null for:
#     - General requests like "Gợi ý lịch trình du lịch"
#     - Questions about specific facilities only
#     - When purpose is not explicitly stated
#     - Mixed or unclear purposes
# - Number_of_people: Extract the number of people or return null if not specified.
# - Companions: Extract the companions mentioned in the request and translated it if it needed, must be one from this list: {companion_list} or return null if not specified.
# - Transportation: Identify the transportation method mentioned in the request and translated, convert it if it needed, transportation must be one from this list: {transport_list} or return null if not specified.
# - Time: Any specific dates or time ranges mentioned or return null if not specified.
# - City: The mentioned city (without "city" or "province").
# - District: The mentioned district (without "district") and must be one frin this list: {district_list} or else return null.
# - Price_range: Specify as "low", "medium", or "high" based on the request.

# **For Hotels, also identify:**
# - Requirements: A summary text of specific requirements or preferences mentioned.
# - Amenities: IMPORTANT - ONLY include amenities from {amenities_list} if EXPLICITLY mentioned in the request. 
#   Examples:
#   - If request says "need hotel with pool and gym" → include ["Pool", "Gym"]
#   - If request doesn't mention any amenities → return null
#   - Do NOT assume or add amenities that weren't specifically mentioned
# - Style: Only include styles from this list if explicitly mentioned in the request: {style_list} or else return null

# **For Restaurants, also identify:**
# - Requirements: A summary text of specific requirements or preferences mentioned.
# - Restaurant_Type: From this list: {restaurant_type_list}
# - Suitable_For: From this list: {suitable_for_list}

# **For Tourist Attractions, also identify:**
# - Requirements: A list of specific requirements or preferences mentioned.
# - Attraction_Type: From this list: {attraction_type_list}

# Return the result using the following JSON format:

# ```json
# {{
#   "General": {{
#     "Type": "...",
#     "Number_of_people": "...",
#     "Companion": "...",
#     "Transportation": "...",
#     "Time": "...",
#     "City": "...",
#     "District": "...",
#     "Price_range": "...",
#     "
#   }},
#   "Hotel": {{
#     "Requirements": ...,
#     "Amenities": [...],
#     "Style": [...]
#   }},
#   "Restaurant": {{
#     "Requirements": ...,
#     "Restaurant_Type": "...",
#     "Suitable_For": "..."
#   }},
#   "TouristAttraction": {{
#     "Attraction_Type": "..."
#   }}
# }}

# ```
# IMPORTANT RULES:
# 1. For lists (Amenities, Style), RETURN null if none are EXPLICITLY mentioned. Do NOT make assumptions or add information that isn't clearly stated or mentioned
# 2. Keep output strictly aligned with the provided lists

# Ensure the JSON is valid. Use null for any unspecified information.
# After the JSON output, add a note in Vietnamese:

# "Nếu bạn cần thay đổi hoặc bổ sung bất kỳ thông tin nào, vui lòng cho tôi biết."
# """

In [ ]:
template = """
You are an AI travel suggestion chatbot. Analyze the following travel request:

Request: "{travel_request}"

Extract general and specific requirements for Hotels, Restaurants, and Tourist Attractions, even if some are not explicitly mentioned. For each type, provide the following information:

**General Requirements:**
- Type:
  - If the request explicitly mentions "nghỉ dưỡng", "thư giãn", "resort", or similar keywords, and the overall tone is relaxed or leisure-oriented or have leisure activities, relaxation-focused activities(clear relaxation keywords), assign "Nghỉ dưỡng".
  - If the request explicitly mentions "khám phá", "văn hóa", "ẩm thực", or similar keywords, and the overall tone is exploratory or adventurous or exploration or have activity-focused activities (clear exploration keywords), assign "Khám phá".
  - For general requests or requests with mixed intentions, return `null`.
- Number_of_people: Extract the number of people or return null if not specified.
- Companions: Extract the companions mentioned in the request and translated it if it needed, must be one from this list: {companion_list} or return null if not specified.
- Transportation: Identify the transportation method mentioned in the request and translated, convert it if it needed, transportation must be one from this list: {transport_list} or return null if not specified.
- Time:
  - Extract specific dates or time ranges mentioned in the request.
  - If no specific dates are mentioned, check for keywords like "ngày", "tuần", "tháng" and their corresponding numbers.
  - Return null if there's no date or time ranges in the request.
  - For example, "3 ngày" should be extracted as "3 days".
- City: The mentioned city (without "city" or "province").
- District: The mentioned district (without "district") and must be one frin this list: {district_list} or else return null.
- Price_range: Specify as "low", "medium", or "high" based on the request.

**For Hotels, also identify:**
- Requirements: A summary text of specific requirements or preferences mentioned.
- Amenities: IMPORTANT - ONLY include amenities from {amenities_list} if EXPLICITLY mentioned in the request. 
  Examples:
  - If request says "need hotel with pool and gym" → include ["Pool", "Gym"]
  - If request doesn't mention any amenities → return null
  - Do NOT assume or add amenities that weren't specifically mentioned
- Style: Only include styles from this list if explicitly mentioned in the request: {style_list} or else return null

**For Restaurants, also identify:**
- Requirements: A summary text of specific requirements or preferences mentioned.
- Restaurant_Type: From this list: {restaurant_type_list}
- Suitable_For: From this list: {suitable_for_list}

**For Tourist Attractions, also identify:**
- Requirements: A list of specific requirements or preferences mentioned.
- Attraction_Type: From this list: {attraction_type_list}

Return the result using the following JSON format:

```json
{{
  "General": {{
    "Type": "...",
    "Number_of_people": "...",
    "Companion": "...",
    "Transportation": "...",
    "Time": "...",
    "City": "...",
    "District": "...",
    "Price_range": "...",
    "
  }},
  "Hotel": {{
    "Requirements": ...,
    "Amenities": [...],
    "Style": [...]
  }},
  "Restaurant": {{
    "Requirements": ...,
    "Restaurant_Type": "...",
    "Suitable_For": "..."
  }},
  "TouristAttraction": {{
    "Attraction_Type": "..."
  }}
}}

```
IMPORTANT RULES:
1. For lists (Amenities, Style), RETURN null if none are EXPLICITLY mentioned. Do NOT make assumptions or add information that isn't clearly stated or mentioned
2. Keep output strictly aligned with the provided lists

Ensure the JSON is valid. Use null for any unspecified information.
After the JSON output, add a note in Vietnamese:

"Nếu bạn cần thay đổi hoặc bổ sung bất kỳ thông tin nào, vui lòng cho tôi biết."
"""

<!-- 1. Use null for ANY field where information is not EXPLICITLY mentioned
2. Do NOT make assumptions or add information that isn't clearly stated
3. When in doubt, return null -->

In [ ]:
prompt = ChatPromptTemplate.from_template(template)
chain = prompt | llm

##### Ask again if there's missing infor

ưu tiên hỏi time

In [ ]:
# ask_template = """
# You are an AI travel suggestion chatbot. Analyze the following travel request:

# Request: "{travel_output_json}"

# ### **Core Rules:**
# 1. ONLY generate questions for fields that meet ALL of the following conditions:
#    - Field value is STRICTLY `null` in the JSON.
#    - Field is marked with *must ask question if this field is null, else not*.

# 2. STRICTLY DO NOT generate questions for:
#    - Fields with any NON-NULL value, even if they are marked with *must ask question if this field is null, else not*.
#    - Fields without *must ask question if this field is null, else not* marking, even if null.

# 3. Special Case for City Validation:
#    - If `"City"` has a value but is not in the `{city_list}`, ask if the user wants to change the city.

# 4. If ALL fields in the `General` section are NON-NULL:
#    - Ask the user if they want to add additional requirements for hotels, restaurants, or tourist attractions.

# 5. Questions about `"Time"` and `"Type"` must be asked first if these fields are null
# ---

# ### **Verification Process:**
# 1. For the `General` section:
#    - **Type**: *must ask question if this field is null, else not* Generate question ONLY if `Type` is `null`.
#    - **Number_of_people**: *must ask question if this field is null, else not* Generate question ONLY if `Number_of_people` is `null`
#    - **Companion**: *must ask question if this field is null, else not* Generate question ONLY if `Companion` is `null`.
#    - **Transportation**: *must ask question if this field is null, else not* Generate question ONLY if `Transportation` is `null`.
#    - **Time**: *must ask question if this field is null, else not* Generate question ONLY if `Time` is `null`.
#    - **Price_range**: *must ask question if this field is null, else not* Generate question ONLY if `Price_range` is `null`.

# 2. For City validation:
#    - If `"City"` is not in `{city_list}` but has a value, ask if the user wants to change it.

# 3. Additional Question (when General is fully completed):
#    - If ALL `General` fields have NON-NULL values, ask:
#      **"Bạn có muốn bổ sung thêm yêu cầu gì cho khách sạn, nhà hàng, hoặc địa điểm tham quan không?"**

# 4. STRICTLY SKIP any field with a non-`null` value.

# ---

# ### **Question Templates (ONLY use if field is NULL AND marked with *must ask question if this field is null, else not*):**
# 1. If `"Type"` is null, ask:  
#    **"Bạn muốn tìm loại hình du lịch nào? (Ví dụ: Food Tour, Văn hóa, Thư giãn, hoặc Trải nghiệm)"**
#    Ignore the question about Type if only ask for one of Hotels, Restaurants, or Tourist Attractions.
   
# 2. If `"Number_of_people"` is null, ask:  
#    **"Bạn đi bao nhiêu người? (Ví dụ: 1, 2, hoặc nhóm lớn hơn)"**

# 3. If `"Companion"` is null, ask:  
#    **"Bạn đi cùng ai? (Bạn bè, Gia đình, hoặc Đồng nghiệp)"**

# 4. If `"Transportation"` is null, ask:  
#    **"Bạn sẽ di chuyển bằng phương tiện gì? (Ví dụ: xe hơi tự lái, xe máy, hoặc phương tiện công cộng)"**

# 5. If `"Time"` is null, ask:  
#    **"Bạn có kế hoạch đi vào thời gian nào không? (Ngày cụ thể hoặc khoảng thời gian)"**

# 6. If `"Price_range"` is null, ask:  
#    **"Bạn muốn ngân sách cho chuyến đi này là bao nhiêu (thấp, trung bình, cao)?"**

# 7. Additional Question (when General is complete):
#    **"Bạn có muốn bổ sung thêm yêu cầu gì cho khách sạn, nhà hàng, hoặc địa điểm tham quan không?"**

# ### **Special Case - City Validation:**
# If City has a value but not in {city_list}:
# **"Hiện tại chúng tôi chưa cung cấp dịch vụ cho thành phố này mà chỉ có tại {city_list}, liệu bạn có muốn thay đổi thành phố không?"**

# ---

# ### **Output Format:**
# 1. Output questions ONLY for fields marked as *must ask question if this field is null, else not* and STRICTLY null.
# 2. Add city validation question if needed.
# 3. If ALL `General` fields are NON-NULL, add the question about additional requirements for hotels, restaurants, or attractions.
# 4. End with: **"Nếu bạn cần thay đổi hoặc bổ sung bất kỳ thông tin nào, vui lòng cho tôi biết."**

# ---

# ### **Example Output:**
# If the JSON input has:
# - `"Transportation"`: `null`
# - `"Time"`: `null`
# - `"City"`: `"Đà Nẵng"` (not in `{city_list}`),
  
# The output will be:

# ```plaintext
# Bạn sẽ di chuyển bằng phương tiện gì? (Ví dụ: xe hơi tự lái, xe máy, hoặc phương tiện công cộng)

# Bạn có kế hoạch đi vào thời gian nào không? (Ngày cụ thể hoặc khoảng thời gian)

# Hiện tại chúng tôi chưa cung cấp dịch vụ cho thành phố này mà chỉ có tại ['Hà Nội'], liệu bạn có muốn thay đổi thành phố không?

# Nếu bạn cần thay đổi hoặc bổ sung bất kỳ thông tin nào, vui lòng cho tôi biết.
# """

In [ ]:
ask_template = """
You are an AI travel assistant chatbot. Analyze the following travel request:

Request: "{travel_output_json}"

### **Core Rules:**

1. **Identify Required Fields:**
    - Identify fields that are marked as "required" in the prompt.
    - If a required field is **null**, generate a question to clarify the user's preference. 

2. **STRICTLY DO NOT** generate questions for:
    - Fields with any **NON-NULL** value.
    - Fields without "required" marking in the prompt.

3. **City Validation:**
    - If the "City" field has a value but is not in the `{city_list}`, ask the user if they want to change the city.

4. **Additional Questions:**
    - Only ask additional questions about hotels, restaurants, or tourist attractions if all required fields in the "General" section have non-null values.

5. Questions about `"Time"` and `"Type"` MUST be ASKED FIRST ONLY if these fields are NULL
---

### **Verification Process:**
1. For the `General` section:
   - **Type** (required): Generate question ONLY if `Type` is `null`.
   - **Number_of_people** (required): Generate question ONLY if `Number_of_people` is `null`
   - **Companion** (required): Generate question ONLY if `Companion` is `null`.
   - **Transportation** (required):Generate question ONLY if `Transportation` is `null`.
   - **Time** (required): Generate question ONLY if `Time` is `null`.
   - **Price_range** (required): Generate question ONLY if `Price_range` is `null`.

2. For City validation:
   - If `"City"` is not in `{city_list}` but has a value, ask if the user wants to change it.

3. Additional Question (when General is fully completed):
   - If ALL required `General` fields have NON-NULL values, ask:
     **"Bạn có muốn bổ sung thêm yêu cầu gì cho khách sạn, nhà hàng, hoặc địa điểm tham quan không?"**

4. STRICTLY SKIP any field with a non-`null` value.

---

### **Question Templates (ONLY use if field is NULL AND marked with *must ask question if this field is null, else not*):**
1. If `"Type"` is null, ask:  
   **"Bạn muốn tìm loại hình du lịch nào? (Ví dụ: Food Tour, Văn hóa, Thư giãn, hoặc Trải nghiệm)"**
   Ignore the question about Type if only ask for one of Hotels, Restaurants, or Tourist Attractions.
   
2. If `"Number_of_people"` is null, ask:  
   **"Bạn đi bao nhiêu người? (Ví dụ: 1, 2, hoặc nhóm lớn hơn)"**

3. If `"Companion"` is null, ask:  
   **"Bạn đi cùng ai? (Bạn bè, Gia đình, hoặc Đồng nghiệp)"**

4. If `"Transportation"` is null, ask:  
   **"Bạn sẽ di chuyển bằng phương tiện gì? (Ví dụ: xe hơi tự lái, xe máy, hoặc phương tiện công cộng)"**

5. If `"Time"` is null, ask:  
   **"Bạn có kế hoạch đi vào thời gian nào không? (Ngày cụ thể hoặc khoảng thời gian)"**

6. If `"Price_range"` is null, ask:  
   **"Bạn muốn ngân sách cho chuyến đi này là bao nhiêu (thấp, trung bình, cao)?"**

7. Additional Question (when General is complete):
   **"Bạn có muốn bổ sung thêm yêu cầu gì cho khách sạn, nhà hàng, hoặc địa điểm tham quan không?"**

### **Special Case - City Validation:**
If City has a value but not in {city_list}:
**"Hiện tại chúng tôi chưa cung cấp dịch vụ cho thành phố này mà chỉ có tại {city_list}, liệu bạn có muốn thay đổi thành phố không?"**

---

### **Output Format:**
1. Output questions ONLY for fields marked as `required` in the prompt and STRICTLY null.
2. Add city validation question if needed.
3. If ALL required `General` fields are NON-NULL, add the question about additional requirements for hotels, restaurants, or attractions.
4. End with: **"Nếu bạn cần thay đổi hoặc bổ sung bất kỳ thông tin nào, vui lòng cho tôi biết."**

---

### **Example Output:**
If the JSON input has:
- `"Transportation"`: `null`
- `"Time"`: `null`
- `"City"`: `"Đà Nẵng"` (not in `{city_list}`),
  
The output will be:

```plaintext
Bạn sẽ di chuyển bằng phương tiện gì? (Ví dụ: xe hơi tự lái, xe máy, hoặc phương tiện công cộng)

Bạn có kế hoạch đi vào thời gian nào không? (Ngày cụ thể hoặc khoảng thời gian)

Hiện tại chúng tôi chưa cung cấp dịch vụ cho thành phố này mà chỉ có tại ['Hà Nội'], liệu bạn có muốn thay đổi thành phố không?

Nếu bạn cần thay đổi hoặc bổ sung bất kỳ thông tin nào, vui lòng cho tôi biết.
"""

<!-- -**Ask only if specific fields are both strictly `null` and marked as *must ask question if this field is null, else not*. Strictly skip the question for any field that has a non-null value, regardless of whether it is marked with *must ask question if this field is null, else not*.**
-**If a field has a value different from `null`, skip the question.**
---

Analyze the JSON to find the fields that have null value. **Only ask questions for fields explicitly marked as *must ask question if this field is null, else not* and set to `null`.** If a populated field is not in the required list, ask if the user would like to change it to a valid option.

**Generate questions for null values in *must ask question if this field is null, else not* fields and if necessary, verify if the City needs adjustment:**
**Skip the question for any field that has a non-null value, regardless of whether it is marked with *must ask question if this field is null, else not*.**
 -->


In [ ]:
ask_prompt = ChatPromptTemplate.from_template(ask_template)
ask_chain = ask_prompt | llm

##### prompt to update requirement json

In [ ]:
updated_query = """
You are an AI travel suggestion chatbot. Analyze the following travel request:

Update request: "{update_travel_request}"

Extract general requirements from request while following these rules:
1. IMPORTANT: Preserve ALL non-null values from the initial request JSON
2. Only update fields that are null in the initial request OR if the city is explicitly changed
3. For new information, extract the following:

**General Requirements:**
- Type: ONLY CLASSIFY INTO TWO TYPES from {travel_type_list}. Analyze the response and map to:
  "Nghỉ dưỡng" if response mentions/implies relaxation-focused activities:
    Keywords to check:
    - "nghỉ mát", "nghỉ dưỡng", "thư giãn"
    - "resort", "spa", "biển"
    - "nghỉ ngơi", "thư thái"
    - "resort", "khách sạn sang trọng"
    Examples:
    - Response: "food tour" -> classify as: "Khám phá"
    - Response: "nghỉ dưỡng" -> classify as: "Nghỉ dưỡng"
    
  "Khám phá" if response mentions/implies exploration and activity-focused activities:
    Keywords to check:
    - "khám phá", "tham quan", "trải nghiệm"
    - "du lịch", "phượt", "tour"
    - "văn hóa", "ẩm thực", "food tour"
    - "địa điểm", "danh lam thắng cảnh"
    Examples:
    - Response: "trải nghiệm văn hóa" -> classify as: "Khám phá"
    - Response: "địa điểm tham quan" -> classify as: "Khám phá"
- Number_of_people: Extract the number of people.
- Companions: Extract the companions mentioned in the request and translated it if it needed, must be one from this list: {companion_list}.
- Transportation: Identify the transportation method mentioned in the request and translated, convert it if it needed, transportation must be one from this list: {transport_list}.
- Time: Any specific dates or time ranges mentioned.
- City: The mentioned city (without "city" or "province") and must be one from this list: {city_list}.
- Price_range: Specify as "low", "medium", or "high" based on the request.

**For Hotels, also identify:**
- Requirements: A summary text of specific requirements or preferences mentioned.
- Amenities: IMPORTANT - ONLY include amenities from {amenities_list} if EXPLICITLY mentioned in the request. 
  Examples:
  - If request says "need hotel with pool and gym" → include ["Pool", "Gym"]
  - If request doesn't mention any amenities → return null
  - Do NOT assume or add amenities that weren't specifically mentioned
- Style: Only include styles from this list if explicitly mentioned in the request: {style_list} or else return null

**For Restaurants, also identify:**
- Requirements: A summary text of specific requirements or preferences mentioned.
- Restaurant_Type: From this list: {restaurant_type_list}
- Suitable_For: From this list: {suitable_for_list}

**For Tourist Attractions, also identify:**
- Requirements: A list of specific requirements or preferences mentioned.
- Attraction_Type: From this list: {attraction_type_list}

Initial request: "{travel_output_json}"

Merge the initial request with any updates, prioritizing:
1. Keeping all non-null values from initial request
2. Only updating null fields or explicitly changed city
3. Using the following JSON format:

```json
{{
  "General": {{
    "Type": "...",
    "Number_of_people": "...",
    "Companion": "...",
    "Transportation": "...",
    "Time": "...",
    "City": "...",
    "District": "...",
    "Price_range": "...",
    "
  }},
  "Hotel": {{
    "Requirements": ...,
    "Amenities": [...],
    "Style": [...]
  }},
  "Restaurant": {{
    "Requirements": ...,
    "Restaurant_Type": "...",
    "Suitable_For": "..."
  }},
  "TouristAttraction": {{
    "Attraction_Type": "..."
  }}
}}

```

IMPORTANT VERIFICATION STEPS:
1. Before outputting, for all other fields, verify that all non-null values from the initial request are preserved unless explicitly changed in the update request.
2. Check if the update is a response to the Type question. If yes, analyze the response using the keyword mapping above
3. Classify into either "Nghỉ dưỡng" or "Khám phá"

Ensure the JSON is valid. Use null for any unspecified information.
After the JSON output, add a note in Vietnamese:

"Nếu bạn cần thay đổi hoặc bổ sung bất kỳ thông tin nào, vui lòng cho tôi biết."
"""

<!-- **For Hotels:**
- Amenities: If in Transportation they give information about "car", add "Bãi đậu xe" to the Amenities. -->

In [ ]:
update_prompt = ChatPromptTemplate.from_template(updated_query)
update_chain = update_prompt | llm